# Projeto Final Awari

Aluno: Armando Carlos de Amorim Nahmias

Mentor: Luciano Santos

Projeto: CoLEX Coleção da Legislação Eleitoral

# Exposição do Problema

O objetivo do presente projeto final é modernizar a divulgação, a gestão e a curadoria de atos normativos da Justiça Eleitoral por meio da análise de dados, para que  qualquer pessoa possa consultar facilmente a legislação eleitoral.

Através do projeto, pretende-se contribuir para o aperfeiçoamento das ferramentas de busca de normas e para a automatização do processo de atualização dos textos normativos. O projeto pode facilitar não só a revisão da legislação eleitoral existente, como também permitir que o TSE, sempre que tenha que editar alguma norma, possa compreender quais são os reflexos dela em toda a legislação eleitoral.

Assim, o questionamento proposto é "Como organizar a legislação eleitoral para otimizar a consulta principalmente de juízes, advogados e servidores da Justiça Eleitoral?"




# Configuração do ambiente

## Importando as bibliotecas necessárias

In [18]:
# Importando a bibliotca de importação
import requests
# Importando a bibliotca de leitura de json
import json
# Importando o pandas para criar o DF
!pip install pandas
import pandas as pd
# Importando o numpy para conferir o tamanho da lista
import numpy as np
# Instalando a biblioteca BeautifulSoup
!pip install beautifulsoup4
# Importando a biblioteca BeautifulSoup
from bs4 import BeautifulSoup
# Importando a biblioteca re
import re

# Coleta dos dados

A coleta de dados corresponderá à coleta primária das Resoluções do TSE do ano de 2019 e 2020.

## Coleta Primária

### Criando funções para coleta

#### Coleta de dados

In [19]:
# Função que busca o conteúdo de uma url fornecida
def endereco(endereco):

    #obtendo a url informada
    conteudo = requests.get(endereco)
    #comunicando eventual erro e abortando
    if conteudo.status_code != 200:
        print('Erro buscando página.')
        exit()
    #obtendo o conteúdo da url informada
    else:
        conteudo = conteudo.content
    
    #devolvendo o resultado
    return conteudo

In [20]:
# Função que coleta os dados das resoluções
def coleta_dados_resolucoes(url):

    #obtendo o conteúdo da url informada
    conteudo = endereco(url)

    #analisando o conteúdo da página buscada
    sopa = BeautifulSoup(conteudo, 'html.parser')

    #extraindo os nomes das resoluções
    set_titulos = sopa.find_all('a', {'class':'state-published'})

    #extraindo as ementas
    set_ementas = sopa.find_all('p', {'style':'text-align: justify; '})

    #extraindo as ligações
    set_ligacoes = [a.get('href') for a in set_titulos]

    #criando listas temporárias
    lista_titulos = []
    lista_ligacoes = []
    lista_ementas = []

    #extraindo apenas o texto
    for titulo, ementa, ligacao in zip(set_titulos, set_ementas, set_ligacoes):
        lista_titulos.append(titulo.text)
        lista_ligacoes.append(ligacao)
        lista_ementas.append(ementa.text)

    #criando um dicionário com os dados requeridos
    dic_dados_resolucoes = {
        'Resolução:': lista_titulos,
        'Ligação:': lista_ligacoes,
        'Ementa:': lista_ementas
    }

    #criando um DF com os dados requeridos
    df_dados_resolucoes = pd.DataFrame(dic_dados_resolucoes)
    

    #devolvendo o resultado
    return df_dados_resolucoes

#### Coleta de textos

In [21]:
# Função que coleta os textos
def coleta_texto_resolucao(url):

    #obtendo o conteúdo da url informada
    conteudo = endereco(url)

    #Analisando o conteúdo da página buscada
    sopa = BeautifulSoup(conteudo, 'html.parser')

    #extraindo o texto da resolucao
    set_texto = sopa.get_text()
 
    #devolvendo o resultado
    return set_texto

### Coletando dados

In [22]:
#Criando uma lista para os dados das Resoluções
dados_resolucoes = []

#extraindo o conteúdo do html
for i in range(2019,2022,1):
    url = f'https://www.tse.jus.br/legislacao/compilada/res/{str(i)}'
    print(f'Obtendo dados da {url}')
    df_dados_resolucoes = coleta_dados_resolucoes(url)
    dados_resolucoes.append(df_dados_resolucoes)


Obtendo dados da https://www.tse.jus.br/legislacao/compilada/res/2019
Obtendo dados da https://www.tse.jus.br/legislacao/compilada/res/2020
Obtendo dados da https://www.tse.jus.br/legislacao/compilada/res/2021


In [23]:
#concatenando as listas criadas
df_dados_resolucoes = pd.concat(dados_resolucoes)

#conferindo os endereços raspados 
ligacoes = df_dados_resolucoes['Ligação:']
ligacoes

0     https://www.tse.jus.br/legislacao/compilada/re...
1     https://www.tse.jus.br/legislacao/compilada/re...
2     https://www.tse.jus.br/legislacao/compilada/re...
3     https://www.tse.jus.br/legislacao/compilada/re...
4     https://www.tse.jus.br/legislacao/compilada/re...
                            ...                        
36    https://www.tse.jus.br/legislacao/compilada/re...
37    https://www.tse.jus.br/legislacao/compilada/re...
38    https://www.tse.jus.br/legislacao/compilada/re...
39    https://www.tse.jus.br/legislacao/compilada/re...
40    https://www.tse.jus.br/legislacao/compilada/re...
Name: Ligação:, Length: 83, dtype: object

In [33]:
#Criando uma lista para os textos das Resoluções
textos_resolucoes = []

#extraindo o conteúdo do html
for i in range(len(ligacoes)):
    url = ligacoes.iloc[i]
    print(f'Obtendo dados da {url}')
    df_dados_resolucoes['Texto:'].iloc(i) = coleta_texto_resolucao(url)
df_dados_resolucoes

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (860919595.py, line 8)

## Infometria dos dados coletados

In [58]:
print("A lista dos dados das Resoluções foi convertida para um quadro com as dimensões:")
print(df_dados_resolucoes.shape)

A lista dos dados das Resoluções foi convertida para um quadro com as dimensões:
(41, 4)


In [59]:
#mostrando o início do quadro criado com os dados da Resoluções
print("As primeiras linhas do quadro de dados coletado são:")
print(df_dados_resolucoes.head())

As primeiras linhas do quadro de dados coletado são:
                                        Resolução:  \
0  RESOLUÇÃO Nº 23.613, DE 28 DE NOVEMBRO DE 2019.   
1  RESOLUÇÃO Nº 23.611, DE 19 DE DEZEMBRO DE 2019.   
2  RESOLUÇÃO Nº 23.610, DE 18 DE DEZEMBRO DE 2019.   
3  RESOLUÇÃO Nº 23.609, DE 18 DE DEZEMBRO DE 2019.   
4  RESOLUÇÃO Nº 23.608, DE 18 DE DEZEMBRO DE 2019.   

                                            Ligação:  \
0  https://www.tse.jus.br/legislacao/compilada/re...   
1  https://www.tse.jus.br/legislacao/compilada/re...   
2  https://www.tse.jus.br/legislacao/compilada/re...   
3  https://www.tse.jus.br/legislacao/compilada/re...   
4  https://www.tse.jus.br/legislacao/compilada/re...   

                                             Ementa:  \
0  Altera a Resolução-TSE nº 21.711, de 6 de abri...   
1  Dispõe sobre os atos gerais do processo eleito...   
2  Dispõe sobre propaganda eleitoral, utilização ...   
3  Dispõe sobre a escolha e o registro de candida...   
4  Di

In [60]:
#exportando os dados raspados para um arquivo csv
df_dados_resolucoes.to_csv('dados_resolucoes.csv', index=False)
#imprimindo o arquivo exportado
pd.read_csv('dados_resolucoes.csv')

,Resolução:,Ligação:,Ementa:,Texto:
0,"RESOLUÇÃO Nº 23.613, DE 28 DE NOVEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Altera a Resolução-TSE nº 21.711, de 6 de abri...","\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
1,"RESOLUÇÃO Nº 23.611, DE 19 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre os atos gerais do processo eleito...,"\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
2,"RESOLUÇÃO Nº 23.610, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre propaganda eleitoral, utilização ...","\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
3,"RESOLUÇÃO Nº 23.609, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre a escolha e o registro de candida...,"\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
4,"RESOLUÇÃO Nº 23.608, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre representações, reclamações e ped...","\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
5,"RESOLUÇÃO Nº 23.607, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre a arrecadação e os gastos de recu...,"\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
6,"RESOLUÇÃO Nº 23.606, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Calendário Eleitoral (Eleições 2020).,"\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
7,"RESOLUÇÃO Nº 23.605, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Estabelece diretrizes gerais para a gestão e d...,"\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
8,"RESOLUÇÃO Nº 23.604, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Regulamenta o disposto no Título III - Das Fin...,"\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
9,"RESOLUÇÃO Nº 23.603, DE 12 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre os procedimentos de fiscalização ...,"\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."


In [61]:
#conferindo a descrição dos dados das resoluções
df_dados_resolucoes.describe()

,Resolução:,Ligação:,Ementa:,Texto:
count,41,41,41,41
unique,41,41,41,1
top,"RESOLUÇÃO Nº 23.613, DE 28 DE NOVEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Altera a Resolução-TSE nº 21.711, de 6 de abri...","\n\n\n\n\n\nRESOLUÇÃO Nº 23.612, DE 8 DE JANEI..."
freq,1,1,1,41


In [62]:
#conferindo as informações sobre os dados das resoluçõse
df_dados_resolucoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 0 to 22
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Resolução:  41 non-null     object
 1   Ligação:    41 non-null     object
 2   Ementa:     41 non-null     object
 3   Texto:      41 non-null     object
dtypes: object(4)
memory usage: 1.6+ KB


### Limpando os dados
limpando os dados

In [75]:
#tirando os parágrafos adicionais
dfna_dados_resolucoes = df_dados_resolucoes.replace(to_replace='\n', value='', regex=True)
dfna_dados_resolucoes.head()

,Resolução:,Ligação:,Ementa:,Texto:
0,"RESOLUÇÃO Nº 23.613, DE 28 DE NOVEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Altera a Resolução-TSE nº 21.711, de 6 de abri...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
1,"RESOLUÇÃO Nº 23.611, DE 19 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre os atos gerais do processo eleito...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
2,"RESOLUÇÃO Nº 23.610, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre propaganda eleitoral, utilização ...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
3,"RESOLUÇÃO Nº 23.609, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre a escolha e o registro de candida...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
4,"RESOLUÇÃO Nº 23.608, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre representações, reclamações e ped...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."


In [76]:
#tirando os espaços adicionais
dfna_dados_resolucoes = dfna_dados_resolucoes.replace(to_replace=' +', value=' ', regex=True)
dfna_dados_resolucoes.head()

,Resolução:,Ligação:,Ementa:,Texto:
0,"RESOLUÇÃO Nº 23.613, DE 28 DE NOVEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Altera a Resolução-TSE nº 21.711, de 6 de abri...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
1,"RESOLUÇÃO Nº 23.611, DE 19 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre os atos gerais do processo eleito...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
2,"RESOLUÇÃO Nº 23.610, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre propaganda eleitoral, utilização ...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
3,"RESOLUÇÃO Nº 23.609, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre a escolha e o registro de candida...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
4,"RESOLUÇÃO Nº 23.608, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre representações, reclamações e ped...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."


In [77]:
#exportando os dados limpos para um arquivo csv
dfna_dados_resolucoes.to_csv('dados_limpos_resolucoes.csv', index=False)
#imprimindo o arquivo exportado
pd.read_csv('dados_limpos_resolucoes.csv')

,Resolução:,Ligação:,Ementa:,Texto:
0,"RESOLUÇÃO Nº 23.613, DE 28 DE NOVEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Altera a Resolução-TSE nº 21.711, de 6 de abri...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
1,"RESOLUÇÃO Nº 23.611, DE 19 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre os atos gerais do processo eleito...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
2,"RESOLUÇÃO Nº 23.610, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre propaganda eleitoral, utilização ...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
3,"RESOLUÇÃO Nº 23.609, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre a escolha e o registro de candida...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
4,"RESOLUÇÃO Nº 23.608, DE 18 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,"Dispõe sobre representações, reclamações e ped...","RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
5,"RESOLUÇÃO Nº 23.607, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre a arrecadação e os gastos de recu...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
6,"RESOLUÇÃO Nº 23.606, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Calendário Eleitoral (Eleições 2020).,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
7,"RESOLUÇÃO Nº 23.605, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Estabelece diretrizes gerais para a gestão e d...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
8,"RESOLUÇÃO Nº 23.604, DE 17 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Regulamenta o disposto no Título III - Das Fin...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
9,"RESOLUÇÃO Nº 23.603, DE 12 DE DEZEMBRO DE 2019.",https://www.tse.jus.br/legislacao/compilada/re...,Dispõe sobre os procedimentos de fiscalização ...,"RESOLUÇÃO Nº 23.612, DE 8 DE JANEIRO DE 2020. ..."
